In [1]:
# ! pip install kgforge

In [2]:
from kgforge.core import KnowledgeGraphForge

In [3]:
forge = KnowledgeGraphForge("../../configurations/demo-forge.yml")

## Imports

In [4]:
from kgforge.core import Resource

In [5]:
from kgforge.specializations.mappings import DictionaryMapping
from kgforge.specializations.mappers import DictionaryMapper

## Source data

In [6]:
scientists = [
    {
        "id": 123,
        "name": "Albert Einstein",
    },
    {
        "id": 456,
        "name": "Peter Higgs",
    },
]

## Use of a published mapping

In [7]:
DATA_SOURCE = "scientists-database"

In [8]:
forge.mappings(DATA_SOURCE)

{'Association': ['DictionaryMapping']}

In [9]:
mapping = forge.mapping("Association", DATA_SOURCE)

In [10]:
resources = forge.map(scientists, mapping, DictionaryMapper)

In [11]:
type(resources)

list

In [12]:
type(resources[0])

kgforge.core.resource.Resource

In [13]:
print(mapping)

{
    id: forge.format("identifier", "associations", x.id)
    type: Association
    agent:
    {
        type: Person
        gender: forge.resolve("male", scope="terms")
        name: x.name
    }
    distribution: forge.attach(f"../../data/scientists-database/{'_'.join(x.name.lower().split())}.txt")
}


In [14]:
print(resources[0])

{
    id: https://kg.example.ch/associations/123
    type: Association
    agent:
    {
        type: Person
        gender:
        {
            id: http://purl.obolibrary.org/obo/PATO_0000384
            label: male
        }
        name: Albert Einstein
    }
    distribution: LazyAction(operation=Store.upload, args=['../../data/scientists-database/albert_einstein.txt'])
}


In [15]:
# forge.register(resources)

## Creation

In [16]:
forge.template("Association")

<info> DemoModel does not distinguish values and constraints in templates for now.
<info> DemoModel does not automatically include nested schemas for now.
{
    type: Association
    agent:
    {
        type: Person
        name: hasattr
    }
}


In [17]:
created = DictionaryMapping("""
    id: forge.format("identifier", "associations", x.id)
    type: Association
    agent:
    {
        type: Person
        name: x.name
        gender: forge.resolve("male", scope="terms")
    }
    distribution: forge.attach(f"../../data/scientists-database/{'_'.join(x.name.lower().split())}.txt")
""")

In [18]:
# created == mapping

In [19]:
resources_bis = forge.map(scientists, created, DictionaryMapper)

In [20]:
resources == resources_bis

True

## Saving

In [21]:
path = "../../mappings/scientists-database/DictionaryMapping/Association.hjson"

In [22]:
created.save(path)

## Loading

In [23]:
loaded = DictionaryMapping.load(path)

In [24]:
# loaded == created